This notebook is to classification Valence data into high (1) and low (0)categories by using video data for each trial which is broken into frames at 1 sec interval

In [ ]:
# import the necessary packages
from keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle

import os

import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images

from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import preprocess_input

import pickle

import sys
import sklearn
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

import tensorflow as tf
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior() 

import time
import math

from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold, train_test_split

#import tensorflow as tf
from keras.models import Model
from keras import layers
from keras import Input
from keras.utils import plot_model

#from tensorflow.keras import layers
from keras.layers import Conv1D, Conv2D, Conv3D, GRU, GlobalAveragePooling1D, Activation, Flatten, Dropout, Dense, MaxPool1D, MaxPool2D, Concatenate, ZeroPadding2D, ZeroPadding1D, Reshape
# import BatchNormalization
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras import backend

# tools for plotting confusion matrices
from matplotlib import pyplot as plt

from keras.optimizers import SGD, Adam

import scipy.io as sio

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Refer to Deap_Video_V2.ipynb to see how the 2 pkl files below were extracted from video data

In [ ]:
with open('/content/drive/My Drive/Deap/X_array_s01.pkl', "wb") as fp:
  pickle.dump( X,fp, protocol=4)

with open('/content/drive/My Drive/Deap/y_s01.pkl', "wb") as fp:
  pickle.dump( dummy_y,fp, protocol=4)

In [ ]:
X.shape

(2398, 576, 720, 3)

In [ ]:
#Our next step is to read the images which we will do based on their names, aka, the Image_ID_Frame column.
image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224)).astype(int)      # reshaping to 224*224*3
    image.append(a)
X = np.array(image)

#before passing any input to the model, we must preprocess it as per the model’s requirement. Otherwise, the model will not perform well enough. Use the preprocess_input() function of keras.applications.vgg16 to perform this step.
from keras.applications.vgg16 import preprocess_input
X = preprocess_input(X, mode='tf')      # preprocessing the input data



In [ ]:
# Validate the shape
X.shape

(2398, 224, 224, 3)

In [ ]:
y.shape[0]

2398

In [ ]:
# SPlit dataset into training-test

msk = np.random.rand(y.shape[0]) < 0.8

data_train = X[msk]
y_train=y[msk]
data_test = X[~msk]
y_test=y[~msk]

In [ ]:
data_test.shape

(493, 224, 224, 3)

In [ ]:
y_test.shape

(493, 2)

In [ ]:
data_train.shape

(1905, 224, 224, 3)

In [ ]:
y_train.shape

(1333, 2)

In [ ]:
print ("now train-test split data")

X_train, X_valid, y_train, y_valid = train_test_split(data_train, y_train, test_size=0.3, random_state=42)    # preparing the validation set

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))    # include_top=False to remove the top layer

print ("predict")

#We will make predictions using this model for X_train and X_valid, get the features, and then use those features to retrain the model.
X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
X_test = base_model.predict(data_test)
X_train.shape, X_valid.shape, X_test.shape

now train-test split data
predict


((1333, 7, 7, 512), (572, 7, 7, 512), (493, 7, 7, 512))

In [ ]:
#The shape of X_train and X_valid is (1333, 7, 7, 512), (572, 7, 7, 512) respectively. In order to pass it to our neural network, we have to reshape it to 1-D.

X_train = X_train.reshape(X_train.shape[0], 7*7*512)      # converting to 1-D
X_valid = X_valid.reshape(X_valid.shape[0], 7*7*512)
X_test= X_test.reshape(X_test.shape[0], 7*7*512)

# We will now preprocess the images and make them zero-centered which helps the model to converge faster.

X_train = X_train/X_train.max()      # centering the data
X_valid = X_valid/X_train.max()
X_test = X_test/X_test.max() 

# Finally, we will build our model. This step can be divided into 3 sub-steps:
# i. Building the model
model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dense(units=1024, activation='sigmoid')) # hidden layer
model.add(Dense(2, activation='softmax'))    # output layer

model.summary()

#We have a hidden layer with 1,024 neurons and an output layer with 2 neurons (since we have 2 classes to predict). Now we will compile our model:

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# In the final step, we will fit the model and simultaneously also check its performance on the unseen images, i.e., validation images:
print ("fitting")
#Training the model
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid))

test_loss, test_acc = model.evaluate(X_test, y_test)
print('test_acc:', test_acc)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 2050      
Total params: 25,693,186
Trainable params: 25,693,186
Non-trainable params: 0
_________________________________________________________________
fitting
Train on 1333 samples, validate on 572 samples
Epoch 1/100
1333/1333 [==============================] - 1s 467us/step - loss: 0.7391 - accuracy: 0.6579 - val_loss: 1.0796 - val_accuracy: 0.7343
Epoch 2/100
1333/1333 [==============================] - 1s 394us/step - loss: 0.4724 - accuracy: 0.7854 - val_loss: 1.0944 - val_accuracy: 0.7692
Epoch 3/100
1333/1333 [==============================] - 1s 385us/step - loss: 0.4265 - accuracy: 0.8080 - val_loss: 1.1295 - val_accuracy: 0.7797
Epoch 4/

Issues encountered in Google Collab with above code:

The above code frequently gave error "Your session crashed after using all available RAM. Hence had to restart with higher RAM".

Another frequently found error was "A Google Drive timeout has occurred" 

In [ ]:
y_test_predicted = model.predict(X_test)

In [ ]:
y_test_predicted.shape

(493, 2)